In [2]:
%run imports_and_functions.ipynb

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Engineering Las Vegas Businesses Dataframe, Indexed by Latitude then Longitude

In [6]:
def dropduplicate_yelp_entries(df) :
    dups = df[df[['latitude','longitude','name']].duplicated(keep=False)].sort_values('latitude')
    mask = dups[['latitude','longitude','name']].duplicated(keep='first')
#     print(dups)
    for name in dups[mask]['name'].values :
        dupset = dups[dups['name']==name].sort_values('review_count', ascending=False)
        dropindex = dupset.iloc[1:,0].index
        df.drop(dropindex, axis=0, inplace=True)

In [3]:
df = pd.read_csv('LasVegasBusinessesFinal.csv', compression='gzip', sep=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12882 entries, 0 to 12881
Data columns (total 20 columns):
lat_index        12882 non-null float64
long_index       12882 non-null float64
lat_index.1      12882 non-null float64
long_index.1     12882 non-null float64
name             12882 non-null object
address          12470 non-null object
attributes       11131 non-null object
categories       12851 non-null object
city             12882 non-null object
latitude         12882 non-null float64
longitude        12882 non-null float64
neighborhood     10489 non-null object
postal_code      12882 non-null int64
review_count     12882 non-null int64
stars            12882 non-null float64
state            12882 non-null object
price            12882 non-null int64
main_category    12845 non-null object
distance         12882 non-null float64
alias            12882 non-null object
dtypes: float64(8), int64(3), object(9)
memory usage: 2.0+ MB


In [5]:
df['price'].value_counts(normalize=True)

0    0.440615
2    0.298556
1    0.160534
3    0.070408
4    0.029887
Name: price, dtype: float64

In [15]:
df.loc[:,'postal_code'] = df['postal_code'].astype(int, copy=False)

In [96]:
df.tail()

,address,attributes,categories,city,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,price
28860,4655 Quality Ct,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Kitchen & Bath, Building Supplies, Home Servic...",Las Vegas,36.105866,-115.206215,Impressions Tile & Marble,NaN,89103,6,4.5,NV,0
28861,"3863 S Valley View Blvd, Ste 31",{'BusinessAcceptsCreditCards': 'True'},"Automotive, Body Shops, Auto Repair, Oil Chang...",Las Vegas,36.120743,-115.192017,Vegas Auto Repair,NaN,89103,9,4.5,NV,0
28862,4322 W Flamingo Rd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Beauty & Spas, Nail Salons, Active Life, Gyms,...",Las Vegas,36.114304,-115.193871,Recharge Salon & Fitness,NaN,89103,20,4.5,NV,3
28863,6166 W Flamingo Rd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Beauty & Spas, Nail Salons, Hair Removal, Waxi...",Las Vegas,36.115367,-115.226848,Aloha Nails,NaN,89103,76,3.5,NV,2
28864,"3900 Paradise Rd, Ste R","{'BikeParking': 'True', 'BusinessAcceptsCredit...","Makeup Artists, Men's Clothing, Swimwear, Shop...",Las Vegas,36.118697,-115.154270,Tito Jimy Salon and Boutique,Eastside,89169,8,4.0,NV,2


In [27]:
df.loc[~(df['postal_code'].isin(vegas_zipcodes)),'postal_code'] = 0

In [93]:
def getPriceRange(string) :
    regex = re.compile("RestaurantsPriceRange2\\': \\'(\d)")
    try :
        match = regex.search(string)
        return int(match.groups(0)[0])
    except : return 0

In [94]:
getPriceRange(df.loc[4,'attributes'])

1

In [95]:
df['price'] = df['attributes'].map(getPriceRange)
df['price'].value_counts()

0    14236
2     8041
1     4798
3     1317
4      473
Name: price, dtype: int64

In [20]:
def get_main_category(catstring) :
    string = ''
    try :
        for cat in main_categories :
            if cat in catstring : string = string + cat + ', '
    except : pass
    return string[0:(len(string)-2)]

In [21]:
get_main_category(df['categories'][0])

'Shopping'

In [22]:
df['main_category'] = df['categories'].map(get_main_category)
df['main_category'].value_counts()

Restaurants                                                                                                3115
Shopping                                                                                                   2467
Home Services                                                                                              2181
Beauty & Spas                                                                                              2060
Health & Medical                                                                                           2016
Food, Restaurants                                                                                          1884
Automotive                                                                                                 1484
Food                                                                                                       1176
Local Services                                                                                          

In [5]:
df[df[['latitude','longitude']].duplicated(keep=False)].sort_values('latitude')

,lat_index,long_index,lat_index.1,long_index.1,name,address,attributes,categories,city,latitude,longitude,name.1,neighborhood,postal_code,review_count,stars,state,price,main_category,distance,lat_index.1.1,long_index.1.1
0,36.058,-115.182086,36.058,-115.182086,Best Mattress,7220 Dean Martin Dr,"{'BikeParking': 'True', 'BusinessAcceptsBitcoi...","Shopping, Furniture Stores, Mattresses, Home &...",Las Vegas,36.058244,-115.182086,Best Mattress,Southwest,89118,25,4.0,NV,2,Shopping,6992.422197,36.058,-115.182
1,36.058,-115.182086,36.058,-115.182086,Silvercar,"7220 Dean Martin Dr, Ste 600",NaN,"Hotels & Travel, Car Rental",Las Vegas,36.058244,-115.182086,Silvercar,Southwest,89118,32,3.5,NV,0,Hotels & Travel,6992.422197,36.058,-115.182
6,36.060,-115.178779,36.060,-115.178779,MaxVegas Club Crawls,7120 Rafael Ridge Way,"{'BusinessAcceptsBitcoin': 'False', 'BusinessA...","Tours, Bars, Party Bus Rentals, Dance Clubs, N...",Las Vegas,36.060033,-115.178779,MaxVegas Club Crawls,Southeast,89119,46,4.5,NV,0,"Event Planning & Services, Hotels & Travel, Ni...",6841.110081,36.060,-115.179
7,36.060,-115.178779,36.060,-115.178779,Sting Alarm,7120 Rafael Ridge Way,"{'BusinessAcceptsBitcoin': 'False', 'BusinessA...","Home Services, Security Services, Security Sys...",Las Vegas,36.060033,-115.178779,Sting Alarm,Southeast,89119,33,3.5,NV,0,"Home Services, Professional Services",6841.110081,36.060,-115.179
15,36.063,-115.223798,36.063,-115.223798,Budget Blinds,"6040 W Badura Ave, Ste 100","{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Shutters, Shades & Blinds",Las Vegas,36.063224,-115.223798,Budget Blinds,Spring Valley,89118,31,4.0,NV,0,Home Services,6994.740285,36.063,-115.224
17,36.063,-115.223798,36.063,-115.223798,Health Binge,"6040 Badura Ave, Ste 140","{'Alcohol': 'none', 'Ambience': ""{'romantic': ...","Restaurants, Food, Juice Bars & Smoothies, Wra...",Las Vegas,36.063224,-115.223798,Health Binge,Spring Valley,89118,22,3.5,NV,2,"Active Life, Food, Restaurants",6994.740285,36.063,-115.224
34,36.066,-115.172243,36.066,-115.172243,Eden In Love,"6671 S Las Vegas Blvd, Ste A-123","{'BikeParking': 'True', 'BusinessAcceptsCredit...","Fashion, Home Decor, Home & Garden, Shopping, ...",Las Vegas,36.066193,-115.172243,Eden In Love,Southeast,89119,7,4.5,NV,2,Shopping,6309.446468,36.066,-115.172
35,36.066,-115.172243,36.066,-115.172243,Ken Blumberg - eXp Realty,6671 S Las Vegas Blvd,NaN,"Real Estate Services, Real Estate Agents, Home...",Las Vegas,36.066193,-115.172243,Ken Blumberg - eXp Realty,Southeast,89119,5,5.0,NV,0,Home Services,6309.446468,36.066,-115.172
38,36.066,-115.172226,36.066,-115.172226,Smoothie King,6649 Las Vegas Blvd S E 100,"{'BikeParking': 'False', 'BusinessAcceptsCredi...","Juice Bars & Smoothies, Food, Health Markets, ...",Las Vegas,36.066499,-115.172226,Smoothie King,Southeast,89119,4,3.0,NV,2,Food,6277.231761,36.066,-115.172
36,36.066,-115.172226,36.066,-115.172226,Hot Dog On A Stick,"6649 Las Vegas Blvd S, Ste 100","{'Ambience': ""{'romantic': False, 'intimate': ...","Fast Food, American (Traditional), Hot Dogs, R...",Las Vegas,36.066499,-115.172226,Hot Dog On A Stick,Southeast,89119,15,4.0,NV,1,"Food, Restaurants",6277.231761,36.066,-115.172


In [7]:
print(df.shape)
dropduplicate_yelp_entries(df)
df.shape

(12882, 22)


(12882, 22)

In [24]:
# df.to_csv('LasVegasBusinessesFinal.csv',sep=';',header=True,index=False,compression='gzip')

In [15]:
df = pd.read_csv('LasVegasBusinessesFinalold.csv',sep=';',compression='gzip')

In [51]:
testlat = 36.120743
testlong = -115.192017 # same coords as autorepair shop in tail of las vegas dataset

In [52]:
df['distance'] = calcDistances(testlat, testlong, df)

maxradius = 7000
df[df['distance']<maxradius].shape

(12882, 20)

In [54]:
# reducing Las Vegas radius due to making manageable total calcDist processing
rowstodrop = df[(df['distance']>maxradius) | df['latitude'].isnull() | df['longitude'].isnull()].index

df.drop(rowstodrop, axis=0, inplace=True)

In [56]:
df['lat_index'] = round(df['latitude'], 3)
df['long_index'] = df['longitude']

In [10]:
df = df.set_index(['lat_index','long_index'], drop=False).sort_index()

In [58]:
df.head()

lat_index  long_index  \
lat_index long_index                           
36.058    -115.182086     36.058 -115.182086   
          -115.182086     36.058 -115.182086   
36.060    -115.202407     36.060 -115.202407   
          -115.182419     36.060 -115.182419   
          -115.182402     36.060 -115.182402   

                                                        name  \
lat_index long_index                                           
36.058    -115.182086                          Best Mattress   
          -115.182086                              Silvercar   
36.060    -115.202407             Guaranteed Carpet Care LLC   
          -115.182419  Sunpower by Renewable Energy Electric   
          -115.182402          The Specialists Detail Studio   

                                            address  \
lat_index long_index                                  
36.058    -115.182086           7220 Dean Martin Dr   
          -115.182086  7220 Dean Martin Dr, Ste 600   
36.060    -115.202407                7135 Rogers St   
          -115.182419  7180 Dean Martin Dr, Ste 100   
          -115.182402  7180 Dean Martin Dr, Ste 300   

                                                              attributes  \
lat_index long_index                                                       
36.058    -115.182086  {'BikeParking': 'True', 'BusinessAcceptsBitcoi...   
          -115.182086                                                NaN   
36.060    -115.202407             {'BusinessAcceptsCreditCards': 'True'}   
          -115.182419  {'BusinessAcceptsCreditCards': 'True', 'ByAppo...   
          -115.182402             {'BusinessAcceptsCreditCards': 'True'}   

                                                              categories  \
lat_index long_index                                                       
36.058    -115.182086  Shopping, Furniture Stores, Mattresses, Home &...   
          -115.182086                        Hotels & Travel, Car Rental   
36.060    -115.202407  Professional Services, Local Services, Carpet ...   
          -115.182419    Electricians, Solar Installation, Home Services   
          -115.182402  Auto Detailing, Wheel & Rim Repair, Car Wash, ...   

                            city   latitude   longitude  \
lat_index long_index                                      
36.058    -115.182086  Las Vegas  36.058244 -115.182086   
          -115.182086  Las Vegas  36.058244 -115.182086   
36.060    -115.202407  Las Vegas  36.060047 -115.202407   
          -115.182419  Las Vegas  36.059594 -115.182419   
          -115.182402  Las Vegas  36.059722 -115.182402   

                                                      name.1 neighborhood  \
lat_index long_index                                                        
36.058    -115.182086                          Best Mattress    Southwest   
          -115.182086                              Silvercar    Southwest   
36.060    -115.202407             Guaranteed Carpet Care LLC    Southwest   
          -115.182419  Sunpower by Renewable Energy Electric    Southwest   
          -115.182402          The Specialists Detail Studio    Southwest   

                       postal_code  review_count  stars state  price  \
lat_index long_index                                                   
36.058    -115.182086        89118            25    4.0    NV      2   
          -115.182086        89118            32    3.5    NV      0   
36.060    -115.202407        89118             9    2.5    NV      0   
          -115.182419        89118            14    1.5    NV      0   
          -115.182402        89118            37    4.5    NV      0   

                                               main_category     distance  \
lat_index long_index                                                        
36.058    -115.182086                               Shopping  6992.422197   
          -115.182086                        Hotels & Travel  6992.422197   
36.060    -115.202

In [61]:
df = df.sort_index(level = [0,1])
df.to_csv('LasVegasBusinessesFinal.csv',sep=';',header=True,index=True,compression='gzip')

### Census Block/Blockgroup Processing for Population/Income

In [7]:
def cut(geoid) :
    index = str(geoid)
    return int(index[0:(len(index)-3)])
cut(320030057022000)

320030057022

In [8]:
def matchincome(blockgroup) :
    try : return blockgroupincome.loc[blockgroup,'income']
    except : return 0

In [9]:
def matchpop(blockgroup) :
    try : return blockgrouppop.loc[blockgroup,'population']
    except : return 0

In [11]:
populations = pd.read_csv('tabblock2010_32_pophu.csv')
populations.columns = ['block','GEO.id2','population']
populations['blockgroup'] = populations['GEO.id2'].map(cut)
populations.set_index(['GEO.id2'],inplace=True)
populations.sort_index()

,block,population,blockgroup
GEO.id2,,,
320019501001000,1000,0,320019501001
320019501001001,1001,0,320019501001
320019501001002,1002,0,320019501001
320019501001003,1003,0,320019501001
320019501001004,1004,0,320019501001
320019501001005,1005,0,320019501001
320019501001006,1006,0,320019501001
320019501001007,1007,0,320019501001
320019501001008,1008,0,320019501001


In [14]:
populations['income'] = populations['blockgroup'].map(matchincome)
populations.tail()

,block,population,blockgroup,income
GEO.id2,,,,
325100010021034,1034,0,325100010021,0
325100003003034,3034,26,325100003003,0
325100005022005,2005,81,325100005022,0
325100003002098,2098,0,325100003002,0
325100007021009,1009,0,325100007021,0


In [12]:
blockgroupincome = pd.read_csv('ClarkCountyIncome.csv', index_col='GEO.id2', usecols=[1,3])
blockgroupincome.columns = ['income']
blockgroupincome.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1291 entries, 320030001011 to 320030078003
Data columns (total 1 columns):
income    1291 non-null object
dtypes: object(1)
memory usage: 20.2+ KB


In [21]:
blockgrouppop = pd.read_csv('ClarkCountyPop.csv', index_col='GEO.id2', usecols=[1,3])
blockgrouppop.columns = ['population']
blockgrouppop.sum()

population    2112436
dtype: int64

In [22]:
populations['blockgrouppop'] = populations['blockgroup'].map(matchpop)
populations.tail()

,block,population,blockgroup,income,blockgrouppop
GEO.id2,,,,,
325100010021034,1034,0,325100010021,0,0
325100003003034,3034,26,325100003003,0,0
325100005022005,2005,81,325100005022,0,0
325100003002098,2098,0,325100003002,0,0
325100007021009,1009,0,325100007021,0,0


### Census Block Processing, Calculate Distance to Block 

In [1]:
features = ['distance','urban','latitude','longitude','blockgroup','income',
                'blockgrouppop','population','landarea','waterarea','block']

In [88]:
def blocksdropna(df) :
    df.drop(df[df['income']==0].index, inplace=True)
    df.drop(df[df['income']=='-'].index, inplace=True)
    df.drop(df[df['population']<20].index, inplace=True)  # min 20 people for block to be counted

In [8]:
blocks = pd.read_csv('Las.csv',sep=';',compression='gzip')
blocks.head()

,Unnamed: 0,GEOID10,INTPTLAT10,INTPTLON10,UR10,ALAND10,AWATER10
0,0,370370204021014,35.750768,-79.465309,U,1834383,8537
1,1,370370208001031,35.737608,-79.195237,R,1207684,4076
2,2,370370207021055,35.732410,-78.963128,R,2316525,30428
3,3,370370202002044,35.745998,-79.385533,R,873908,7445
4,4,370370202001004,35.829309,-79.469416,R,5128921,49040


In [9]:
blocks.columns = ['block','GEO.id2','urban','landarea','waterarea','latitude','longitude']
blocks['blockgroup'] = blocks['GEO.id2'].map(cut)
blocks = blocks.set_index('GEO.id2')
blocks.sort_index()

NameError: name 'cut' is not defined

In [76]:
blocks = blocks.join(populations.loc[:,['income','blockgrouppop','population']], how='inner')

In [89]:
blocksdropna(blocks)
blocks.shape

(1231, 11)

In [90]:
blocks[['blockgrouppop','population']].sum()

blockgrouppop    1728763
population        208050
dtype: int64

In [172]:
blocks[(blocks['population']!=0) & (blocks['income']==0)] # must drop as don't have y target (income)

,block,urban,landarea,waterarea,latitude,longitude,blockgroup,income,blockgrouppop,population
GEO.id2,,,,,,,,,,
320030024033000,3000,U,225343,0,36.128166,-115.150390,320030024033,0,18,520
320030057031022,1022,R,46826549,0,35.748407,-115.284956,320030057031,0,14,150
320030067004002,4002,U,42787,0,36.117026,-115.156401,320030067004,0,11,20


In [173]:
blocks[(blocks['population']!=0) & (blocks['blockgrouppop']==0)]

,block,urban,landarea,waterarea,latitude,longitude,blockgroup,income,blockgrouppop,population
GEO.id2,,,,,,,,,,
320030038004000,4000,U,23640,0,36.195029,-115.128999,320030038004,-,0,10
320030038004001,4001,U,24154,0,36.195301,-115.130714,320030038004,-,0,6
320030038004003,4003,U,73567,0,36.193800,-115.132565,320030038004,-,0,5
320030060015005,5005,U,25262,0,36.248036,-115.050882,320030060015,-,0,55


In [180]:
blocks[(blocks['population']!=0) & (blocks['income']=='-')] # must drop as don't have y target (income)

,block,urban,landarea,waterarea,latitude,longitude,blockgroup,income,blockgrouppop,population,distance
GEO.id2,,,,,,,,,,,
320030038004000,4000,U,23640,0,36.195029,-115.128999,320030038004,-,0,10,8780.292064
320030038004001,4001,U,24154,0,36.195301,-115.130714,320030038004,-,0,6,8761.306822
320030038004003,4003,U,73567,0,36.193800,-115.132565,320030038004,-,0,5,8553.026643
320030060015005,5005,U,25262,0,36.248036,-115.050882,320030060015,-,0,55,17268.210579


In [48]:
# search = SearchEngine(simple_zipcode=True)
# zipcode = search.by_zipcode("89109")
testlat, testlong  # used as center of Las Vegas

(36.120743, -115.192017)

In [79]:
blocks['distance'] = blocks.apply(
    (lambda row : calcDist(row['latitude'],row['longitude'],testlat, testlong)),         
    axis=1)

In [80]:
blocks.shape

(16386, 11)

In [84]:
blocks = blocks[blocks['distance']<6000]
blocks.shape

(1335, 11)

In [85]:
blocks = blocks.reindex(['distance','urban','latitude','longitude','blockgroup','income',
                'blockgrouppop','population','landarea','waterarea','block'], axis=1)
blocks.head()

,distance,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea,block
GEO.id2,,,,,,,,,,,
320030029581005,5403.219921,U,36.093640,-115.241871,320030029581,25757,1495,155,54924,0,1005
320030029572000,5808.613932,U,36.096091,-115.248927,320030029572,52256,884,437,728659,0,2000
320030010042004,3855.731439,U,36.155053,-115.198804,320030010042,25408,1249,32,17103,0,2004
320030010031016,4532.868829,U,36.154625,-115.220151,320030010031,24759,1058,29,41920,0,1016
320030029381020,3136.357472,U,36.137785,-115.219814,320030029381,34314,1539,35,39561,0,1020


In [86]:
blocks[blocks['distance']<6000][['population']].sum()

population    209558
dtype: int64

### Bootstrapping Rows for Blocks df to match Block Population

In [91]:
def bootstrapBlocks(blocksdf) :
    df = blocksdf.copy()
    print(df.index)
    for index in blocksdf[blocksdf['population']>200].index :
        iterpop = int(np.ceil(blocksdf.loc[index,'population'] / 200))
        lis = [pd.Series(data=blocksdf.loc[index].values, name=index+(10**15*i),
                        index=blocksdf.columns) for i in range(1,iterpop)]
#         print(len(lis))
        df = df.append(lis) # more efficient and faster to append all at once
    return df

In [92]:
320030067003000+10**15
blocks.shape

(1231, 11)

In [93]:
blocks = bootstrapBlocks(blocks)
blocks.shape

Int64Index([320030029581005, 320030029572000, 320030010042004, 320030010031016,
            320030029381020, 320030022032005, 320030029561001, 320030010045005,
            320030026033000, 320030029381014,
            ...
            320030029472021, 320030029491005, 320030029371001, 320030029392014,
            320030001032008, 320030029421001, 320030024061000, 320030012003014,
            320030029682005, 320030029411010],
           dtype='int64', name='GEO.id2', length=1231)


(1784, 11)

In [94]:
blocks.to_csv('LasVegasTrainRowCoords.csv',
          sep=';',
          header=True,
          index=True,
          compression='gzip')

### Latitude, Longitude, Calculate Distances and Zipcodes (is no long used) 

In [ ]:
df = pd.read_csv('LasVegasBusinessesFinal.csv',sep=';',compression='gzip')

In [28]:
def calcDistances(lat, long, citydf) :
    distances = pd.Series(index=citydf.index)
    for index in citydf.index :
        distances[index] = calcDist(lat, long, citydf.loc[index,'latitude'], citydf.loc[index,'longitude'])
    return distances

In [29]:
def latlong_tozipcode(tupl) :
    try :
        (lat, long) = tupl
        result = search.by_coordinates(lat, long, radius=10, returns=1)
        int(result[0].to_dict()['zipcode'])
    except : return 0
    return int(result[0].to_dict()['zipcode'])

In [40]:
results = calcDistances(testlat, testlong, df)
results[results < search_radius]

124      756.578301
208      765.864667
255      973.197697
414      460.796078
569      738.852141
608      558.352126
702      515.163792
748      470.980523
978      942.223872
1054     780.714841
1085     757.099413
1316     625.965896
1405     682.566558
1419     923.452452
1616     727.021330
1799     807.826330
1800     112.244559
1837     774.976823
1998     500.465278
2041     555.086623
2052     842.593902
2189     938.324328
2241      81.875111
2311     111.693278
2374     738.819004
2400     549.937323
2444     646.253775
2445     808.289048
2602     845.699750
3009     610.566873
3177     269.861606
3194     594.179987
3251     652.673844
3561     793.966080
3566     780.592592
3694     593.037600
3896     734.244402
3908     702.917976
4026     492.257434
4030     475.775473
4117     799.044674
4251     394.917450
4340     916.518562
4467     626.201874
4591     684.898624
4592     671.926328
4607     474.757591
4716     551.012449
4767     947.815840
4900     895.142118


In [125]:
results.sort_values(ascending=False)

15878    7.994029e+06
17333    3.393894e+06
25418    3.160668e+06
7846     3.088254e+06
26826    9.011564e+05
12289    6.390267e+05
12154    6.110386e+05
24917    3.780776e+05
12056    3.704614e+05
10905    3.672562e+05
7585     3.616097e+05
6066     3.370235e+05
21509    3.168293e+05
12003    2.750879e+05
241      1.549299e+05
25088    1.435822e+05
3104     1.259141e+05
3352     7.895170e+04
22929    7.191085e+04
19800    6.126371e+04
5795     5.971009e+04
8628     5.955960e+04
7507     5.615010e+04
27010    5.005020e+04
19841    4.899625e+04
11509    4.832166e+04
6759     4.832166e+04
17134    4.796614e+04
12204    4.530206e+04
20853    4.530206e+04
12584    4.530206e+04
23935    4.530206e+04
4167     4.530206e+04
28755    4.530206e+04
21263    4.530206e+04
11656    4.530206e+04
19219    4.530206e+04
9523     4.530206e+04
10485    4.530206e+04
10565    4.530206e+04
22320    4.530206e+04
12643    4.530206e+04
9458     4.504370e+04
28118    4.498598e+04
6215     4.419634e+04
28028    4

In [41]:
results[results < search_radius].shape

(555,)

In [43]:
df.loc[results[results < search_radius].index,'price'].value_counts()

0    225
2    185
1     98
3     39
4      8
Name: price, dtype: int64

In [46]:
df['postal_code'].value_counts()  # 74 unique zip codes plus 0 for mislabeled or missing zipcodes

89109    3116
89119    1808
89102    1618
89103    1483
89117    1376
89118    1340
89123    1206
89146    1182
89101    1182
89147    1116
89104     981
89121     880
89128     855
89148     727
89130     647
89113     636
89139     614
89120     603
89108     561
89145     536
89135     524
89169     512
89149     510
89106     484
89129     440
89183     423
0         420
89107     415
89131     340
89115     340
89110     315
89134     259
89122     230
89141     180
89144     169
89178     140
89142     101
89136     101
89156      64
89143      56
89158      55
89166      50
89138      37
89161      27
89154      25
89124      23
89162      21
89044      20
89111      18
89133      10
89191       9
89193       9
89179       9
89105       7
89155       5
89180       5
89140       4
89112       4
89199       4
89157       3
89165       3
89152       3
89153       3
89137       3
89151       3
89160       2
89114       2
89054       2
89173       2
89116       2
89126       1
89125 

In [104]:
vegas_zipcodes = [
89044,89054,89101,89102,89103,89104,89105,89106,89107,89108,89109,89110,89111,89112,89113,89114,
89115,89116,89117,89118,89119,89120,89121,89122,89123,89124,89125,89126,89127,89128,89129,89130,
89131,89132,89133,89134,89135,89136,89137,89138,89139,89140,89141,89142,89143,89144,89145,89146,
89147,89148,89149,89150,89151,89152,89153,89154,89155,89156,89157,89158,89159,89160,89161,89162,
89163,89164,89165,89166,89169,89170,89173,89177,89178,89179,89180,89183,89185,89191,89193,89195,89199]
# From https://data.mongabay.com/igapo/zip_codes/NV.htm#i5DwGpyATaUgxp84.99

In [53]:
Ydata = pd.read_csv('lasvegaszipcodes.csv')
Ydata

,#,Zip Code,Location,City,Population,Avg. Income/H/hold
0,1,89141,"35.982470, -115.205227","Las Vegas, Nevada",262,"$108,579.00"
1,2,89148,"36.063771, -115.315877","Las Vegas, Nevada",913,"$92,707.00"
2,3,89139,"36.032308, -115.208632","Las Vegas, Nevada","2,297","$74,706.00"
3,4,89149,"36.279319, -115.300319","Las Vegas, Nevada","11,312","$74,482.00"
4,5,89144,"36.171202, -115.345319","Las Vegas, Nevada","11,772","$70,136.00"
5,6,89143,"36.659049, -115.185865","Las Vegas, Nevada","2,408","$67,411.00"
6,7,89129,"36.235825, -115.298399","Las Vegas, Nevada","32,441","$63,373.00"
7,8,89135,"36.119441, -115.323855","Las Vegas, Nevada","3,539","$62,629.00"
8,9,89131,"36.294631, -115.235476","Las Vegas, Nevada","10,968","$61,779.00"
9,10,89113,"36.060558, -115.263553","Las Vegas, Nevada","8,276","$61,057.00"


In [99]:
bad_zipcodes = set(df['postal_code']).difference(set(Ydata['Zip Code']))
# the remaining zip codes are just for PO boxes, physical location of these businesses must be
# determined by their lat, long

In [42]:
df.tail()

,address,attributes,categories,city,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,price,main_category
28860,4655 Quality Ct,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Kitchen & Bath, Building Supplies, Home Servic...",Las Vegas,36.105866,-115.206215,Impressions Tile & Marble,NaN,89103,6,4.5,NV,0,"Home Services, Shopping"
28861,"3863 S Valley View Blvd, Ste 31",{'BusinessAcceptsCreditCards': 'True'},"Automotive, Body Shops, Auto Repair, Oil Chang...",Las Vegas,36.120743,-115.192017,Vegas Auto Repair,NaN,89103,9,4.5,NV,0,Automotive
28862,4322 W Flamingo Rd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Beauty & Spas, Nail Salons, Active Life, Gyms,...",Las Vegas,36.114304,-115.193871,Recharge Salon & Fitness,NaN,89103,20,4.5,NV,3,"Active Life, Beauty & Spas"
28863,6166 W Flamingo Rd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Beauty & Spas, Nail Salons, Hair Removal, Waxi...",Las Vegas,36.115367,-115.226848,Aloha Nails,NaN,89103,76,3.5,NV,2,Beauty & Spas
28864,"3900 Paradise Rd, Ste R","{'BikeParking': 'True', 'BusinessAcceptsCredit...","Makeup Artists, Men's Clothing, Swimwear, Shop...",Las Vegas,36.118697,-115.154269,Tito Jimy Salon and Boutique,Eastside,89169,8,4.0,NV,2,"Beauty & Spas, Shopping"


In [98]:
set(t).difference(set(df['postal_code']))

{21201,
 29936,
 30461,
 89002,
 89011,
 89012,
 89014,
 89015,
 89019,
 89030,
 89031,
 89032,
 89048,
 89052,
 89074,
 89081,
 89084,
 89085,
 89086,
 90014,
 90720,
 90831,
 91016,
 92647,
 94546,
 95133}

In [96]:
t = list(map(latlong_tozipcode, zip(df['latitude'], df['longitude'])))

In [97]:
df['zipcode'] = list(map(latlong_tozipcode, zip(df['latitude'], df['longitude'])))

In [105]:
df[~df['zipcode'].isin(set(vegas_zipcodes))][['postal_code','zipcode']]

,postal_code,zipcode
3,0,89031
28,89119,89014
87,0,89030
90,89119,89014
92,0,89031
96,0,89032
129,89101,89032
131,0,89012
241,0,0
244,0,89030


In [103]:
df['zipcode'].value_counts()

89109    2847
89103    1821
89101    1661
89102    1649
89147    1421
89146    1351
89123    1300
89119    1212
89169    1121
89128    1109
89118     930
89117     906
89113     869
89107     843
89145     798
89120     748
89121     700
89130     685
89144     658
89149     598
89104     576
89148     563
89106     521
89110     507
89183     480
89108     459
89139     343
89122     238
89134     226
89115     223
89129     191
89014     177
89141     145
89179     144
89085     118
89143     116
89030     110
89031      77
89138      64
89135      57
89052      56
89032      34
0          28
89166      23
89074      21
89131      20
89124      19
89081      15
89142      12
89012      11
89054      11
89084      11
89011       8
89086       7
89015       6
89161       5
89178       2
89019       2
89002       1
90014       1
91016       1
29936       1
30461       1
89048       1
90720       1
94546       1
21201       1
90831       1
95133       1
92647       1
Name: zipcode, dtype

In [108]:
results.sort_values(ascending=False)[:200]

15878    7.994029e+06
17333    3.393894e+06
25418    3.160668e+06
7846     3.088254e+06
26826    9.011564e+05
12289    6.390267e+05
12154    6.110386e+05
24917    3.780776e+05
12056    3.704614e+05
10905    3.672562e+05
7585     3.616097e+05
6066     3.370235e+05
21509    3.168293e+05
12003    2.750879e+05
241      1.549299e+05
25088    1.435822e+05
3104     1.259141e+05
3352     7.895170e+04
22929    7.191085e+04
19800    6.126371e+04
5795     5.971009e+04
8628     5.955960e+04
7507     5.615010e+04
27010    5.005020e+04
19841    4.899625e+04
11509    4.832166e+04
6759     4.832166e+04
17134    4.796614e+04
12204    4.530206e+04
20853    4.530206e+04
12584    4.530206e+04
23935    4.530206e+04
4167     4.530206e+04
28755    4.530206e+04
21263    4.530206e+04
11656    4.530206e+04
19219    4.530206e+04
9523     4.530206e+04
10485    4.530206e+04
10565    4.530206e+04
22320    4.530206e+04
12643    4.530206e+04
9458     4.504370e+04
28118    4.498598e+04
6215     4.419634e+04
28028    4

In [69]:
search = SearchEngine(simple_zipcode=True)

In [66]:
result = search.by_coordinates(39.122229, -77.133578, radius=10, returns=1)

In [68]:
result[0].to_dict()

{'zipcode': '20855',
 'zipcode_type': 'Standard',
 'major_city': 'Derwood',
 'post_office_city': 'Derwood, MD',
 'common_city_list': ['Derwood', 'Rockville'],
 'county': 'Montgomery County',
 'state': 'MD',
 'lat': 39.14,
 'lng': -77.13,
 'timezone': 'Eastern',
 'radius_in_miles': 4.0,
 'area_code_list': ['240', '301'],
 'population': 14295,
 'population_density': 1090.0,
 'land_area_in_sqmi': 13.11,
 'water_area_in_sqmi': 0.25,
 'housing_units': 5018,
 'occupied_housing_units': 4889,
 'median_home_value': 453900,
 'median_household_income': 120603,
 'bounds_west': -77.177635,
 'bounds_east': -77.099876,
 'bounds_north': 39.18584,
 'bounds_south': 39.101573}

### Writing csv files for US Cities from Kaggle's Yelp Business Dataset

In [2]:
test = pd.read_json('yelp_academic_dataset_business.json', lines=True)
test.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",MontrÃ©al,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,5,4.0,QC
3,211 W Monroe St,None,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Phoenix,None,1,33.449999,-112.076979,Geico Insurance,,85003,8,1.5,AZ
4,2005 Alyth Place SE,{'BusinessAcceptsCreditCards': 'True'},8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Calgary,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,51.035591,-114.027366,Action Engine,,T2H 0N5,4,2.0,AB


In [6]:
test.iloc[0,3]

'Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel'

In [5]:
test.drop('hours', axis=1, inplace=True)

In [7]:
test.drop('is_open', axis=1, inplace=True)

In [8]:
test.drop('business_id', axis=1, inplace=True)

In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188593 entries, 0 to 188592
Data columns (total 12 columns):
address         188593 non-null object
attributes      162807 non-null object
categories      188052 non-null object
city            188593 non-null object
latitude        188587 non-null float64
longitude       188587 non-null float64
name            188593 non-null object
neighborhood    188593 non-null object
postal_code     188593 non-null object
review_count    188593 non-null int64
stars           188593 non-null float64
state           188593 non-null object
dtypes: float64(3), int64(1), object(8)
memory usage: 17.3+ MB


In [16]:
test[test['city']=='Las Vegas'].to_csv('LasVegasBusinesses.csv',
                                      sep=';',
                                      header=True,
                                      index=False,
                                      compression='gzip')

In [17]:
test[test['city']=='Phoenix'].to_csv('PhoenixBusinesses.csv',
                                      sep=';',
                                      header=True,
                                      index=False,
                                      compression='gzip')

In [18]:
test[test['city']=='Charlotte'].to_csv('CharlotteBusinesses.csv',
                                      sep=';',
                                      header=True,
                                      index=False,
                                      compression='gzip')

In [19]:
test[test['city']=='Pittsburgh'].to_csv('PittsburghBusinesses.csv',
                                      sep=';',
                                      header=True,
                                      index=False,
                                      compression='gzip')

### Business Density Plot by Jagan Gupta from Kaggle

In [ ]:
# Code from https://www.kaggle.com/jagangupta for Kaggle's yelp challenge in
# https://www.kaggle.com/jagangupta/what-s-in-a-review-yelp-ratings-eda/notebook


import imageio
import folium
import folium.plugins as plugins
from mpl_toolkits.basemap import Basemap

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,7))

#a random point inside vegas
lat = 36.207430
lon = -115.268460
#some adjustments to get the right pic
lon_min, lon_max = lon-0.3,lon+0.5
lat_min, lat_max = lat-0.4,lat+0.5
#subset for vegas
ratings_data_vegas=rating_data[(rating_data["longitude"]>lon_min) &\
                    (rating_data["longitude"]<lon_max) &\
                    (rating_data["latitude"]>lat_min) &\
                    (rating_data["latitude"]<lat_max)]

#Facet scatter plot
ratings_data_vegas.plot(kind='scatter', x='longitude', y='latitude',
                color='yellow', 
                s=.02, alpha=.6, subplots=True, ax=ax1)
ax1.set_title("Las Vegas")
ax1.set_facecolor('black')

#a random point inside pheonix
lat = 33.435463
lon = -112.006989
#some adjustments to get the right pic
lon_min, lon_max = lon-0.3,lon+0.5
lat_min, lat_max = lat-0.4,lat+0.5
#subset for pheonix
ratings_data_pheonix=rating_data[(rating_data["longitude"]>lon_min) &\
                    (rating_data["longitude"]<lon_max) &\
                    (rating_data["latitude"]>lat_min) &\
                    (rating_data["latitude"]<lat_max)]
#plot pheonix
ratings_data_pheonix.plot(kind='scatter', x='longitude', y='latitude',
                color='yellow', 
                s=.02, alpha=.6, subplots=True, ax=ax2)
ax2.set_title("Pheonix")
ax2.set_facecolor('black')
f.show()

In [ ]:
data=[]
#rearranging data to suit the format needed for folium
stars_list=list(rating_data['stars'].unique())
for star in stars_list:
    subset=ratings_data_vegas[ratings_data_vegas['stars']==star]
    data.append(subset[['latitude','longitude']].values.tolist())
#initialize at vegas
lat = 36.127430
lon = -115.138460
zoom_start=11
print("                     Vegas Review heatmap Animation ")

# basic map
m = folium.Map(location=[lat, lon], tiles="OpenStreetMap", zoom_start=zoom_start)
#inprovising the Heatmapwith time plugin to show variations across star ratings 
hm = plugins.HeatMapWithTime(data,max_opacity=0.3,auto_play=True,display_index=True,radius=7)
hm.add_to(m)
m